# Project 3

In [2]:
%pip install pandas mlxtend


Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

filename = "luxury_dealership_sales_data.csv"
chunksize = 3000
df_list = []

for chunk in pd.read_csv(filename, chunksize=chunksize):
    chunk['StoreID'] = chunk['StoreID'].astype('int32')
    chunk['CustomerID'] = chunk['CustomerID'].astype('int32')
    chunk['OrderID'] = chunk['OrderID'].astype('int32')
    chunk['Price'] = chunk['Price'].astype('float32')
    chunk['Car'] = chunk['Car Brand & Model'] + ' ' + chunk['Paint Color']
    df_list.append(chunk)

df = pd.concat(df_list)


# Top 10 Products Most Frequently Bought Together

In [16]:
from mlxtend.frequent_patterns import apriori, association_rules

# Sample a portion of the dataset for analysis
sampled_df = df.sample(frac=0.1, random_state=1)

# Group and transform the data for basket analysis
basket = sampled_df.groupby(['OrderID', 'Car'])['Price'].count().unstack().reset_index().fillna(0).set_index('OrderID')
basket = basket.map(lambda x: 1 if x > 0 else 0).astype(bool)

# Perform Apriori algorithm to find frequent itemsets
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)
num_itemsets = len(frequent_itemsets)

# Generate association rules from the frequent itemsets
rules = association_rules(frequent_itemsets, num_itemsets=num_itemsets, metric="lift", min_threshold=1)

# Get the top 10 products frequently bought together
top_10_products = rules.sort_values('lift', ascending=False).head(10)
print("Top 10 selling products most frequently bought together:")
print(top_10_products[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


Top 10 selling products most frequently bought together:
                      antecedents                  consequents   support  \
896                (BMW SUV Grey)       (Mercedes Sedan White)  0.012779   
897        (Mercedes Sedan White)               (BMW SUV Grey)  0.012779   
241         (BMW Convertible Red)           (Porsche SUV Grey)  0.013890   
240            (Porsche SUV Grey)        (BMW Convertible Red)  0.013890   
1548     (Lexus Convertible Grey)  (Porsche Convertible White)  0.013890   
1549  (Porsche Convertible White)     (Lexus Convertible Grey)  0.013890   
937             (Lexus SUV Black)                (BMW SUV Red)  0.012001   
936                 (BMW SUV Red)            (Lexus SUV Black)  0.012001   
1474    (Lexus Convertible Black)           (Mercedes SUV Red)  0.013779   
1475           (Mercedes SUV Red)    (Lexus Convertible Black)  0.013779   

      confidence      lift  
896     0.140244  1.318762  
897     0.120167  1.318762  
241     0.119503  1

# Top 10 Stores and their Top 10 Products

In [14]:


# Add a 'Units Sold' column
df['Units Sold'] = 1

# Group by StoreID and Car to calculate total sales and units sold
store_sales = df.groupby(['StoreID', 'Car']).agg({'Price': 'sum', 'Units Sold': 'sum'}).reset_index()

# Identify top 10 stores
top_10_stores = store_sales.groupby('StoreID')['Price'].sum().nlargest(10).index

# Initialize a DataFrame to store results
results = pd.DataFrame()

# For each of the top 10 stores, identify their top 10 products
for store in top_10_stores:
    top_products = store_sales[store_sales['StoreID'] == store].sort_values('Price', ascending=False).head(10)
    top_products['StoreID'] = store  # Ensure StoreID is included in the result
    results = pd.concat([results, top_products])

# Reset index for clear output
results.reset_index(drop=True, inplace=True)

# Print results
for store in top_10_stores:
    store_data = results[results['StoreID'] == store]
    print(f"\nTop 10 products in Store {store}:")
    print(store_data[['StoreID', 'Car', 'Price', 'Units Sold']])



Top 10 products in Store 1397:
   StoreID                         Car       Price  Units Sold
0     1397              BMW SUV Silver  48673916.0         585
1     1397     Lexus Convertible Black  29523440.0         331
2     1397  Mercedes Convertible Black  21148492.0         179
3     1397          Mercedes Coupe Red  20367160.0         220
4     1397        BMW Convertible Grey  19017680.0         191
5     1397           Porsche Coupe Red  18204800.0         200
6     1397         Porsche Sedan White  17707788.0         177
7     1397    Lexus Convertible Silver  16180595.0         197
8     1397           Lexus Sedan White  15992416.0         176
9     1397           Lexus Coupe White  15617175.0         197

Top 10 products in Store 135:
    StoreID                         Car       Price  Units Sold
10      135  Mercedes Convertible White  28118000.0         358
11      135       BMW Convertible White  24514420.0         299
12      135   Porsche Convertible Black  19443144.0 

# Top Selling Cars by Paints, Models and Brand

In [13]:


# Ensure 'Car Brand & Model' column is a string
df['Car Brand & Model'] = df['Car Brand & Model'].astype(str)

# Correctly combine 'Car Brand & Model' into separate 'Brand' and 'Model' columns
df[['Brand', 'Model']] = df['Car Brand & Model'].str.split(n=1, expand=True)

# Add a 'Units Sold' column
df['Units Sold'] = 1

# Top selling paint colors
top_paint_colors = df.groupby('Paint Color').agg({'Price': 'sum', 'Units Sold': 'sum'}).sort_values(by='Price', ascending=False).reset_index()
print("Top Selling Paint Colors:")
print(top_paint_colors)

# Top selling models
top_models = df.groupby('Model').agg({'Price': 'sum', 'Units Sold': 'sum'}).sort_values(by='Price', ascending=False).reset_index()
print("\nTop Selling Models:")
print(top_models)

# Top selling brands
top_brands = df.groupby('Brand').agg({'Price': 'sum', 'Units Sold': 'sum'}).sort_values(by='Price', ascending=False).reset_index()
print("\nTop Selling Brands:")
print(top_brands)


Top Selling Paint Colors:
  Paint Color         Price  Units Sold
0         Red  1.297248e+10      172184
1      Silver  1.290216e+10      173065
2       White  1.270779e+10      170377
3        Grey  1.257524e+10      166965
4       Black  1.245057e+10      166348

Top Selling Models:
         Model         Price  Units Sold
0        Sedan  1.651966e+10      220097
1  Convertible  1.623887e+10      215125
2        Coupe  1.577192e+10      210126
3          SUV  1.507779e+10      203591

Top Selling Brands:
      Brand         Price  Units Sold
0   Porsche  1.607393e+10      218754
1  Mercedes  1.594908e+10      212150
2     Lexus  1.586597e+10      210333
3       BMW  1.571925e+10      207702


# Top Selling Products Overall

In [15]:
# Overall top-selling items across the organization
overall_top_products = df.groupby('Car').agg({'Price': 'sum', 'Units Sold': 'sum'}).sort_values(by='Price', ascending=False).reset_index()
print("\nOverall Top-Selling Products:")
print(overall_top_products.head(10))



Overall Top-Selling Products:
                          Car        Price  Units Sold
0     Porsche Convertible Red  988905408.0       12677
1           Porsche Sedan Red  975689024.0       13061
2          Porsche SUV Silver  969110400.0       13762
3         Porsche Sedan Black  958791424.0       12655
4  Mercedes Convertible White  932755072.0       12228
5   Mercedes Convertible Grey  928466112.0       12873
6          Porsche Sedan Grey  927173056.0       12136
7            BMW Coupe Silver  915352384.0       11381
8     Lexus Convertible White  900087744.0       11737
9             BMW Sedan White  899476160.0       12458
